In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import manifold
from sklearn.manifold import TSNE
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import NullFormatter
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt

from time import time

In [2]:
df_train = pd.read_csv('train_genelevel_90%.tsv', sep='\t',index_col=0)
#df.set_index(df["Unnamed: 0"], inplace=True)
df_train.shape

(55169, 407)

In [3]:
df_train

,205_2,251_1,474_3,178_4,313_1,484_4,288_1,106_4,215_2,184_3,...,330_2,11_4,480_1,314_1,371_3,443_2,311_1,225_2,150_4,289_2
ENSG00000000419,202.0,243.00,230.0,270.0,232.00,231.00,174.00,164.00,244.0,207.0,...,257.00,274.00,230.00,327.00,252.0,285.0,237.00,223.0,278.00,159.0
ENSG00000000457,261.0,369.00,355.0,375.0,317.00,273.00,254.00,198.00,229.0,344.0,...,335.00,297.00,348.00,389.00,461.0,272.0,419.00,376.0,312.00,380.0
ENSG00000000460,194.0,188.00,234.0,236.0,385.00,154.00,144.00,253.00,218.0,214.0,...,331.00,198.00,192.00,274.00,251.0,322.0,364.00,129.0,322.00,252.0
ENSG00000000938,25.0,15.00,14.0,14.0,14.00,14.00,10.00,15.00,10.0,20.0,...,20.00,18.00,43.00,85.00,13.0,8.0,17.00,14.0,23.00,2.0
ENSG00000000971,428.0,91.06,83.0,184.0,167.78,138.79,105.76,196.97,236.0,61.0,...,118.77,163.48,200.57,266.55,175.0,344.0,146.84,67.0,367.51,97.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000283693,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,...,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0
ENSG00000283695,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,...,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0
ENSG00000283696,5.0,1.00,1.0,5.0,1.00,1.00,0.00,2.00,3.0,3.0,...,3.00,0.00,2.00,4.00,1.0,2.0,5.00,0.0,1.00,4.0
ENSG00000283698,1.0,1.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,1.0,...,0.00,0.00,1.00,0.00,0.0,0.0,0.00,1.0,0.00,0.0


# Normalize across the column (by patient)

In [4]:
## get the sum for the column
sum_col = df_train.sum(axis=0)
sum_col

205_2    11339776.43
251_1    11474174.19
474_3    11493209.25
178_4    13767712.58
313_1    10777905.28
            ...     
443_2    11284345.75
311_1    11414900.39
225_2    10615704.62
150_4    10083242.19
289_2     8009566.15
Length: 407, dtype: float64

In [5]:
## sum across the column, then divide by each library depth
new_df = df_train.div(df_train.sum(axis=0), axis=1)
new_df

,205_2,251_1,474_3,178_4,313_1,484_4,288_1,106_4,215_2,184_3,...,330_2,11_4,480_1,314_1,371_3,443_2,311_1,225_2,150_4,289_2
ENSG00000000419,1.781340e-05,2.117799e-05,2.001182e-05,1.961110e-05,2.152552e-05,2.148744e-05,0.000022,1.835732e-05,2.528430e-05,1.616092e-05,...,1.662916e-05,0.000028,2.295356e-05,2.679032e-05,1.904284e-05,2.525623e-05,2.076234e-05,2.100661e-05,2.757050e-05,1.985126e-05
ENSG00000000457,2.301633e-05,3.215918e-05,3.088780e-05,2.723764e-05,2.941202e-05,2.539425e-05,0.000032,2.216310e-05,2.372994e-05,2.685679e-05,...,2.167614e-05,0.000031,3.472973e-05,3.186983e-05,3.483630e-05,2.410419e-05,3.670641e-05,3.541922e-05,3.094243e-05,4.744327e-05
ENSG00000000460,1.710792e-05,1.638462e-05,2.035985e-05,1.714155e-05,3.572123e-05,1.432496e-05,0.000018,2.831952e-05,2.259007e-05,1.670742e-05,...,2.141732e-05,0.000020,1.916123e-05,2.244816e-05,1.896727e-05,2.853511e-05,3.188815e-05,1.215181e-05,3.193417e-05,3.146238e-05
ENSG00000000938,2.204629e-06,1.307284e-06,1.218111e-06,1.016872e-06,1.298954e-06,1.302269e-06,0.000001,1.679023e-06,1.036242e-06,1.561441e-06,...,1.294098e-06,0.000002,4.291318e-06,6.963845e-06,9.823685e-07,7.089467e-07,1.489282e-06,1.318801e-06,2.281012e-06,2.497014e-07
ENSG00000000971,3.774325e-05,7.936083e-06,7.221656e-06,1.336460e-05,1.556703e-05,1.291014e-05,0.000013,2.204781e-05,2.445531e-05,4.762396e-06,...,7.685000e-06,0.000017,2.001650e-05,2.183780e-05,1.322419e-05,3.048471e-05,1.286389e-05,6.311404e-06,3.644760e-05,1.211052e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000283693,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
ENSG00000283695,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
ENSG00000283696,4.409258e-07,8.715224e-08,8.700790e-08,3.631685e-07,9.278241e-08,9.301923e-08,0.000000,2.238697e-07,3.108726e-07,2.342162e-07,...,1.941147e-07,0.000000,1.995962e-07,3.277103e-07,7.556681e-08,1.772367e-07,4.380240e-07,0.000000e+00,9.917445e-08,4.994028e-07
ENSG00000283698,8.818516e-08,8.715224e-08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,7.807206e-08,...,0.000000e+00,0.000000,9.979808e-08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.420006e-08,0.000000e+00,0.000000e+00


In [8]:
## get the maximum of library depth
max_ld = df_train.sum(axis=0).max()
max_ld

34678509.490000084

In [9]:
## multiply by the largest library depth value
new_df2 = new_df.multiply(max_ld)
new_df2

,205_2,251_1,474_3,178_4,313_1,484_4,288_1,106_4,215_2,184_3,...,330_2,11_4,480_1,314_1,371_3,443_2,311_1,225_2,150_4,289_2
ENSG00000000419,617.742242,734.421290,693.979985,680.083747,746.472899,745.152502,757.493166,636.604390,876.821887,560.436479,...,576.674369,982.341697,795.995208,929.048373,660.377140,875.848314,720.006874,728.478033,956.103747,688.412194
ENSG00000000457,798.171907,1115.232329,1071.143020,944.560760,1019.965125,880.634775,1105.765886,768.583349,822.918902,931.353377,...,751.696162,1064.801036,1204.375359,1105.198217,1208.070880,835.897339,1272.923546,1228.285831,1073.037299,1645.261848
ENSG00000000460,593.277203,568.194249,706.049202,594.443572,1238.758906,496.768335,626.890896,982.078724,783.390047,579.388438,...,742.720685,709.867358,664.482956,778.468667,657.756596,989.554938,1105.833343,421.406575,1107.429520,1091.068383
ENSG00000000938,76.453248,45.334648,42.242260,35.263602,45.045778,45.160758,43.534090,58.226011,35.935323,54.148452,...,44.877383,64.533396,148.816495,241.495754,34.067075,24.585216,51.646063,45.734047,79.102109,8.659273
ENSG00000000971,1308.879602,275.211533,250.436255,463.464480,539.841479,447.704397,460.416536,764.585163,848.073628,165.152779,...,266.504338,586.106645,694.142430,757.302275,458.595236,1057.164281,446.100462,218.870082,1263.948518,419.974735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000283693,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000283695,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000283696,15.290650,3.022310,3.017304,12.594143,3.217556,3.225768,0.000000,7.763468,10.780597,8.122268,...,6.731607,0.000000,6.921697,11.364506,2.620544,6.146304,15.190018,0.000000,3.439222,17.318546
ENSG00000283698,3.058130,3.022310,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.707423,...,0.000000,0.000000,3.460849,0.000000,0.000000,0.000000,0.000000,3.266718,0.000000,0.000000


# Standardize across the row (by gene) --> Mean 0 and Sd 1

In [10]:
## get the mean across the row
ave = new_df2.mean(axis=1)
ave

ENSG00000000419     746.910154
ENSG00000000457    1003.378633
ENSG00000000460     742.286326
ENSG00000000938      53.096386
ENSG00000000971     554.751181
                      ...     
ENSG00000283693       0.010047
ENSG00000283695       0.000000
ENSG00000283696       7.991246
ENSG00000283698       0.610334
ENSG00000283699       0.000000
Length: 55169, dtype: float64

In [11]:
## get the standard deviation across the row
sig = new_df2.std(axis=1)
sig

ENSG00000000419    165.449058
ENSG00000000457    250.143257
ENSG00000000460    216.917723
ENSG00000000938     28.687375
ENSG00000000971    283.774796
                      ...    
ENSG00000283693      0.144145
ENSG00000283695      0.000000
ENSG00000283696      5.881928
ENSG00000283698      1.373078
ENSG00000283699      0.000000
Length: 55169, dtype: float64

In [12]:
## get the mean across the row, then subtract by the mean for each row
df_sub = new_df2.sub(new_df2.mean(axis=1), axis=0)
df_sub

,205_2,251_1,474_3,178_4,313_1,484_4,288_1,106_4,215_2,184_3,...,330_2,11_4,480_1,314_1,371_3,443_2,311_1,225_2,150_4,289_2
ENSG00000000419,-129.167911,-12.488864,-52.930169,-66.826406,-0.437254,-1.757652,10.583013,-110.305763,129.911733,-186.473674,...,-170.235785,235.431543,49.085055,182.138219,-86.533013,128.938161,-26.903279,-18.432121,209.193593,-58.497959
ENSG00000000457,-205.206726,111.853695,67.764387,-58.817873,16.586492,-122.743858,102.387253,-234.795284,-180.459731,-72.025257,...,-251.682471,61.422403,200.996725,101.819584,204.692247,-167.481295,269.544912,224.907197,69.658665,641.883215
ENSG00000000460,-149.009123,-174.092077,-36.237124,-147.842754,496.472580,-245.517991,-115.395430,239.792398,41.103721,-162.897888,...,0.434359,-32.418968,-77.803369,36.182341,-84.529730,247.268612,363.547017,-320.879751,365.143194,348.782058
ENSG00000000938,23.356862,-7.761739,-10.854126,-17.832784,-8.050608,-7.935628,-9.562296,5.129625,-17.161063,1.052066,...,-8.219003,11.437010,95.720109,188.399368,-19.029311,-28.511170,-1.450323,-7.362339,26.005722,-44.437113
ENSG00000000971,754.128421,-279.539648,-304.314926,-91.286702,-14.909703,-107.046784,-94.334646,209.833982,293.322447,-389.598402,...,-288.246844,31.355463,139.391249,202.551093,-96.155945,502.413100,-108.650720,-335.881100,709.197337,-134.776447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000283693,-0.010047,-0.010047,-0.010047,-0.010047,-0.010047,-0.010047,-0.010047,-0.010047,-0.010047,-0.010047,...,-0.010047,-0.010047,-0.010047,-0.010047,-0.010047,-0.010047,-0.010047,-0.010047,-0.010047,-0.010047
ENSG00000283695,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000283696,7.299404,-4.968936,-4.973941,4.602898,-4.773690,-4.765477,-7.991246,-0.227777,2.789351,0.131022,...,-1.259638,-7.991246,-1.069548,3.373261,-5.370701,-1.844942,7.198773,-7.991246,-4.552023,9.327300
ENSG00000283698,2.447796,2.411975,-0.610334,-0.610334,-0.610334,-0.610334,-0.610334,-0.610334,-0.610334,2.097088,...,-0.610334,-0.610334,2.850514,-0.610334,-0.610334,-0.610334,-0.610334,2.656383,-0.610334,-0.610334


In [13]:
## get the sd across the row, then divide by the sd for each row
df_standardize = df_sub.div(df_sub.std(axis=1), axis=0)
df_standardize

,205_2,251_1,474_3,178_4,313_1,484_4,288_1,106_4,215_2,184_3,...,330_2,11_4,480_1,314_1,371_3,443_2,311_1,225_2,150_4,289_2
ENSG00000000419,-0.780711,-0.075485,-0.319918,-0.403909,-0.002643,-0.010624,0.063965,-0.666705,0.785207,-1.127076,...,-1.028932,1.422985,0.296678,1.100872,-0.523019,0.779322,-0.162608,-0.111407,1.264399,-0.353571
ENSG00000000457,-0.820357,0.447159,0.270902,-0.235137,0.066308,-0.490694,0.409314,-0.938643,-0.721426,-0.287936,...,-1.006153,0.245549,0.803526,0.407045,0.818300,-0.669542,1.077562,0.899114,0.278475,2.566062
ENSG00000000460,-0.686938,-0.802572,-0.167055,-0.681561,2.288760,-1.131848,-0.531978,1.105453,0.189490,-0.750966,...,0.002002,-0.149453,-0.358677,0.166802,-0.389686,1.139919,1.675967,-1.479269,1.683326,1.607900
ENSG00000000938,0.814186,-0.270563,-0.378359,-0.621625,-0.280632,-0.276624,-0.333328,0.178811,-0.598210,0.036673,...,-0.286502,0.398677,3.336663,6.567327,-0.663334,-0.993858,-0.050556,-0.256640,0.906522,-1.549013
ENSG00000000971,2.657489,-0.985076,-1.072382,-0.321687,-0.052541,-0.377224,-0.332428,0.739438,1.033645,-1.372914,...,-1.015759,0.110494,0.491204,0.713774,-0.338846,1.770464,-0.382877,-1.183619,2.499155,-0.474942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000283693,-0.069703,-0.069703,-0.069703,-0.069703,-0.069703,-0.069703,-0.069703,-0.069703,-0.069703,-0.069703,...,-0.069703,-0.069703,-0.069703,-0.069703,-0.069703,-0.069703,-0.069703,-0.069703,-0.069703,-0.069703
ENSG00000283695,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000283696,1.240988,-0.844780,-0.845631,0.782549,-0.811586,-0.810190,-1.358610,-0.038725,0.474224,0.022275,...,-0.214154,-1.358610,-0.181836,0.573496,-0.913085,-0.313663,1.223880,-1.358610,-0.773900,1.585756
ENSG00000283698,1.782706,1.756619,-0.444501,-0.444501,-0.444501,-0.444501,-0.444501,-0.444501,-0.444501,1.527290,...,-0.444501,-0.444501,2.076003,-0.444501,-0.444501,-0.444501,-0.444501,1.934619,-0.444501,-0.444501
